In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("titanic.csv")
drop_columns = ["Name","Cabin","Ticket","PassengerId","SibSp","Parch","Age"]
dataset = dataset.drop(drop_columns,axis = 1)

In [3]:
print(dataset.head())
dataset["Embarked"] = dataset["Embarked"].fillna('S')
#dataset["Age"] = dataset["Age"].fillna(dataset["Age"].mean())
print(dataset.info())

   Survived  Pclass     Sex     Fare Embarked
0         0       3    male   7.2500        S
1         1       1  female  71.2833        C
2         1       3  female   7.9250        S
3         1       1  female  53.1000        S
4         0       3    male   8.0500        S
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 34.9+ KB
None


In [4]:
'''
dataset.plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
plt.show()
'''
'''
from pandas.plotting import scatter_matrix
scatter_matrix(dataset)
plt.show()
'''

'\nfrom pandas.plotting import scatter_matrix\nscatter_matrix(dataset)\nplt.show()\n'

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset["Sex"] = le.fit_transform(dataset["Sex"])
dataset["Embarked"] = le.fit_transform(dataset["Embarked"])

print(dataset.head())

   Survived  Pclass  Sex     Fare  Embarked
0         0       3    1   7.2500         2
1         1       1    0  71.2833         0
2         1       3    0   7.9250         2
3         1       1    0  53.1000         2
4         0       3    1   8.0500         2


In [6]:
x_data_columns = ["Pclass","Sex","Fare","Embarked"]
x_data = dataset[x_data_columns]
y_data = dataset["Survived"]


# Decision Tree Implementation

In [7]:
def entropy(col):
    count = np.unique(col,return_counts=True)
    m = col.shape[0]
    ent = 0.0
    for ix in count[1]:
        p = ix/m
        ent -= p*np.log2(p)
    return ent
def divide(data,fkey,fval):
    left = pd.DataFrame([],columns=data.columns)
    right = pd.DataFrame([],columns=data.columns)
    for ix in range(data.shape[0]):
        val = data[fkey].loc[ix]
        if val>fval:
            right = right.append(data.loc[ix])
        else:
            left = left.append(data.loc[ix])
    right = right.reset_index(drop = True)
    left - left.reset_index(drop = True)
    return right,left
def IG(data,fkey,fval):
    right_data,left_data = divide(data,fkey,fval)
    r = float(right_data.shape[0]/data.shape[0])
    l = float(left_data.shape[0]/data.shape[0])
    if left_data.shape[0] == 0 or right_data.shape[0] ==0:
        return -1000000
    infogain = entropy(data.Survived)-(entropy(right_data.Survived)+entropy(left_data.Survived))
    return infogain

In [8]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.depth = depth
        self.max_depth = max_depth
        self.fval = None
        self.fkey = None
        self.target = None
    def train(self,data):
        features = ["Pclass","Sex","Fare","Embarked"]
        info_gain = []
        for ix in features:
            ig = IG(data,ix,data[ix].mean())
            info_gain.append(ig)
        idx = np.argmax(info_gain)
        self.fkey = features[idx]
        self.fval = data[self.fkey].mean()
        print(self.fkey)
        right_data,left_data = divide(data,self.fkey,self.fval)
        left_data = left_data.reset_index(drop=True)
        right_data = right_data.reset_index(drop=True)
        if right_data.shape[0]==0 or left_data.shape[0]==0:
            if data.Survived.mean()>=0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        if (self.depth>=self.max_depth):
            if data.Survived.mean()>=0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        self.left = DecisionTree(depth = self.depth + 1,max_depth = self.max_depth)
        self.left.train(left_data)
        self.right = DecisionTree(depth = self.depth + 1,max_depth = self.max_depth)
        self.right.train(right_data)
        if data.Survived.mean()>=0.5:
            self.target = "Survive"
        else:
            self.target = "Dead"
        return   
    def predict(self,test):
        if test[self.fkey]>self.fval:
            #go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [9]:
split = int(0.7*dataset.shape[0])
train_data = dataset[:split]
test_data = dataset[split:]
test_data = test_data.reset_index(drop=True)

In [10]:
dt = DecisionTree()
dt.train(dataset)
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))
print(y_pred)

Sex
Pclass
Embarked
Embarked
Pclass
Fare
Fare
Pclass
Pclass
Pclass
Pclass
Fare
Fare
Fare
Fare
Fare
Fare
Embarked
Fare
Embarked
Fare
Fare
Fare
Embarked
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Pclass
Pclass
Embarked
Embarked
Fare
Pclass
Fare
Fare
Fare
Fare
Fare
Fare
Embarked
Embarked
Fare
Fare
Embarked
Embarked
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
Fare
['Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Survive', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Survive', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead', 'Dead'

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

predictions = le.fit_transform(y_pred)
Actual = test_data.Survived
cnt = 0
for i in range(Actual.shape[0]):
    if(Actual[i]==predictions[i]):
        cnt+=1
acc = float(cnt/Actual.shape[0])
print(acc)

0.8171641791044776


# Decision Tree Using Sklearn

In [12]:
input_col = ["Pclass","Sex","Fare","Embarked"]
output_col = ["Survived"]
from sklearn.tree import DecisionTreeClassifier
sk_tree = DecisionTreeClassifier(criterion='entropy')
sk_tree.fit(train_data[input_col],train_data[output_col])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
sk_tree.predict(test_data[input_col])
sk_tree.score(test_data[input_col],test_data[output_col])

0.8283582089552238

# Visualization

In [14]:
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

dot_data = StringIO()
export_graphviz(sk_tree,out_file = dot_data,filled = True, rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [ ]:
'''
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(sk_tree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())
'''

# Random Forest Classifier

In [16]:
X_train = train_data[input_col]
Y_train = np.array(train_data[output_col]).reshape((-1,))
X_test = test_data[input_col]
Y_test = np.array(test_data[output_col]).reshape((-1,))
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='entropy', max_depth = 5)
rf.fit(X_train,Y_train)
print(rf.score(X_train,Y_train))
print(rf.score(X_test,Y_test))

0.8443017656500803
0.7985074626865671


C:\Users\MUJ\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
